In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from preprocessing.data.imagenet_labels import imagenet_labels

from modules.ViT_Layers import PatchCNN, ClassToken, Epos, ViT_Block

In [2]:
def decode_fn(record_bytes):
    example = tf.io.parse_single_example(
        # Data
        record_bytes,

        # Schema
        {"image": tf.io.FixedLenFeature([], dtype=tf.string),
         "label": tf.io.FixedLenFeature([], dtype=tf.int64)}
            )
    image = tf.io.parse_tensor(example["image"],
                               out_type = tf.uint8)
    label = example["label"]
    
    return image, label

In [3]:
path = './preprocessing/data/TFRs/train/'
files = [path+n for n in os.listdir(path)]
ds = tf.data.TFRecordDataset(files).map(decode_fn)
print(ds)

<MapDataset shapes: (<unknown>, ()), types: (tf.uint8, tf.int64)>


In [5]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        
        self.PatchCNN = PatchCNN()
        self.ClassToken = ClassToken()
        self.Epos = Epos()
        
        self.Block_1 = ViT_Block()
        self.Block_2 = ViT_Block()
        self.Block_3 = ViT_Block()
        self.Block_4 = ViT_Block()
        
        self.Block_5 = ViT_Block(num_of_classes = 1000,
                                 Last = True)
        
    def call(self, x):
        
        x = tf.ensure_shape(x, [None,128,128,3])
        
        x = self.PatchCNN(x)
        x = self.ClassToken(x)
        x = self.Epos(x)
        #x = self.Epos(self.ClassToken(self.PatchCNN(x)))
        
        x = self.Block_1(x)
        x = self.Block_2(x)
        x = self.Block_3(x)
        x = self.Block_4(x)
        x = self.Block_5(x)
        
        return x

model = MyModel()

In [6]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
model.fit(ds.batch(5),batch_size=5, epochs=10)

Epoch 1/10


ValueError: in user code:

    /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    <ipython-input-5-7510e5800eef>:22 call  *
        x = self.ClassToken(x)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py:1008 __call__  **
        self._maybe_build(inputs)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py:2710 _maybe_build
        self.build(input_shapes)  # pylint:disable=not-callable
    /tf/modules/ViT_Layers.py:49 build
        self.tiled_class_token = tf.tile(class_token, [input_shape[0],1,1])
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gen_array_ops.py:11455 tile
        input, multiples, name=name, ctx=_ctx)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gen_array_ops.py:11491 tile_eager_fallback
        _attr_Tmultiples, (multiples,) = _execute.args_to_matching_eag

In [ ]:
for _ in range(10000):
    with tf.GradientTape() as tape:
        logits = model(images)
        loss_value = loss(logits, tf.one_hot(label,1000))
    grads = tape.gradient(loss_value, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    
    print(loss_value)

In [2]:
%tensorboard --log_dir ./log/

UsageError: Line magic function `%tensorboard` not found.
